In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import pandas as pd

In [ ]:
# get Senate map Soup opbject
on_lower = 'house'
URL = f'https://www.sos.state.nm.us/voting-and-elections/data-and-maps/{on_lower}-district-maps/'
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [ ]:
tables = soup.find('table')
dict_list = []
for i in tables.tbody.findAll('tr'):
    dis_text = i.find_next('td').text
    lnk_text = i.find_all('td')[1].find_next('a')['href']
    if (re.search('[0-9]+', dis_text) is not None): 
        dict_list.append({f'{on_lower}_district':re.findall('[0-9]+', dis_text)[0], 
                       'Map Link':lnk_text})
df_link = pd.DataFrame(dict_list)
df_link.head()
#df_link.dtypes

In [ ]:
# join with legislator file
ld = pd.read_csv(f'./legislator-info/nm_{on_lower}_legislator_info.csv').drop(['Map Link'], axis = 1)
print(ld.head())
print(ld.dtypes)
#ld['District'] = ld['District'].astype('str')
df_link[f'{on_lower}_district'] = df_link[f'{on_lower}_district'].astype('int64')
merged_df = ld.merge(df_link, how = 'left', left_on = 'District', right_on = f'{on_lower}_district').drop([f'{on_lower}_district'], axis = 1)

In [ ]:
merged_df.to_excel('legislator-info/nm-house-legislator-info.xlsx')